Want:

main/control loss for:

- GSAE
- GSAE finetune
- GSAE-phys (sweep L0)
- GSAE + SSAE (sweep L0)
- clean loss

In [1]:
import sys
sys.path.append("/root/specialised-SAEs")
from datasets import load_dataset
from transformer_lens import utils, HookedTransformer
import gc
import torch
from sae_lens.jacob.load_sae_from_hf import load_sae_from_hf
from config import DTYPE_MAP
from tqdm import tqdm

In [2]:
torch.set_grad_enabled(False)
ctx_length = 128
DTYPE = "bfloat16"
model = HookedTransformer.from_pretrained("gemma-2b-it", device="cuda", dtype=DTYPE,
                                          fold_ln= False, 
                                          center_writing_weights = False, 
                                          center_unembed = False,
                                          fold_value_biases = False)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer


In [3]:
gsae = load_sae_from_hf("jacobcd52/gemma2-gsae", 
                        "sae_weights.safetensors", 
                        "cfg.json",
                        device="cuda",
                        dtype=DTYPE)

ssae_list = [
    load_sae_from_hf("jacobcd52/gemma2-ssae-phys",
                    f"l1_coeff={l1_coeff}_tokens=40960000_lr=0.001.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=40960000_lr=0.001_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [5, 10, 20]
]

gsae_0_list = [
    load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 2, 5]
]

gsae_0p1_list = [
    load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.1.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.1_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 2, 5]
]

gsae_0p5_list = [
    load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.5.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.5_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 2, 5]
]

sae_weights.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

GSAE weights file saved as temp_sae/sae_weights.safetensors


cfg.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


(…)f=5_tokens=40960000_lr=0.001.safetensors:   0%|          | 0.00/302M [00:00<?, ?B/s]

...

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


(…)00_lr=0.0001_control_mix=0.5.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

GSAE weights file saved as temp_sae/sae_weights.safetensors


(…)40000_lr=0.0001_control_mix=0.5_cfg.json:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


In [4]:
# get OWT tokens
data = load_dataset("stas/openwebtext-10k", split="train")
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=ctx_length)
tokenized_data = tokenized_data.shuffle(42)
owt_tokens = tokenized_data["tokens"][:20_000].cuda()
print("owt_tokens has shape", owt_tokens.shape)
print("total number of tokens:", int(owt_tokens.numel()//1e6), "million")
print()
# get physics-papers tokens
data = load_dataset("jacobcd52/physics-papers", split="train[:10%]")
# Define a filter function to remove null entries
def remove_null_entries(example):
    return all(value is not None and value != '' for value in example.values())
# Apply the filter to remove null entries
data = data.filter(remove_null_entries)
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=ctx_length)
tokenized_data = tokenized_data.shuffle(42)
phys_tokens = tokenized_data["tokens"][:20_000].cuda()
print("phys_tokens has shape", phys_tokens.shape)
print("total number of tokens:", int(phys_tokens.numel()//1e6), "million")

# clean up
del tokenized_data, data
gc.collect()

Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

owt_tokens has shape torch.Size([20000, 128])
total number of tokens: 2 million



Map (num_proc=10):   0%|          | 0/1570 [00:00<?, ? examples/s]

phys_tokens has shape torch.Size([20000, 128])
total number of tokens: 2 million


0

In [5]:
# sanity check
hook_pt = 'blocks.12.hook_resid_pre'
loss, cache = model.run_with_cache(
    "Hi, my name is Jacob, and I come from the city of London, England.",
    return_type="loss",
    names_filter=[hook_pt]
)

act = cache[hook_pt]
(gsae_0p5_list[0](act) - act).norm(), act.norm()

(tensor(36., device='cuda:0', dtype=torch.bfloat16),
 tensor(692., device='cuda:0', dtype=torch.bfloat16))

In [6]:
def get_l0_loss_mse(
        sae_list, 
        tokens, 
        num_tokens = 100_000, 
        hook_pt = 'blocks.12.hook_resid_pre', 
        batch_size=64
        ):
    
    # define hook fn to patch in SAE reconstructions, as well as cache the L0
    l0_dic = {}
    mse_dic = {}
    def patch_hook(act, hook):              
        # option to run the model clean
        if sae_list == "clean":
            l0_dic[0] = 0
            mse_dic[0] = 0
            return act
        # patch in SAE reconstructions
        else:
            l0 = 0
            out = torch.zeros_like(act)
            for sae in sae_list:
                feature_acts = sae.encode_fn(act)
                l0 += (feature_acts > 0).to(DTYPE_MAP[DTYPE]).sum(dim=-1).mean().item()
                out += sae.decode(feature_acts)

        # save l0 to dictionary and return the output to patch
        l0_dic[0] = l0
        mse_dic[0] = ((out - act) ** 2).sum(dim=-1).mean().item()
        return out
    
    # initialise running variables
    total_l0 = 0
    total_mse = 0
    total_loss = 0

    num_batches = num_tokens // (tokens.shape[1] * batch_size)

    for b in tqdm(range(num_batches)):
        # get batch
        batch = tokens[b*batch_size:(b+1)*batch_size]
        losses = model.run_with_hooks(
            batch,
            return_type="loss",
            loss_per_token=True,
            fwd_hooks = [(hook_pt, patch_hook)]
        )[:, 1:]
        total_l0 += l0_dic[0]
        total_mse += mse_dic[0]
        total_loss += losses.mean().item()
        
    return total_l0 / num_batches, total_loss / num_batches, total_mse / num_batches

In [26]:
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

# Get baseline losses, where we don't need to sweep
_, clean_owt_loss, _ = get_l0_loss_mse("clean", owt_tokens)
_, clean_phys_loss, _ = get_l0_loss_mse("clean", phys_tokens)
gsae_owt_l0, gsae_owt_loss, gsae_owt_mse = get_l0_loss_mse([gsae], owt_tokens)
gsae_phys_l0, gsae_phys_loss, gsae_phys_mse = get_l0_loss_mse([gsae], phys_tokens)

ssae_owt_l0, ssae_owt_loss, ssae_owt_mse = get_l0_loss_mse([ssae_list[-1]], owt_tokens)
ssae_phys_l0, ssae_phys_loss, ssae_phys_mse = get_l0_loss_mse([ssae_list[-1]], phys_tokens)

print(f"clean OWT loss = {clean_owt_loss:.3f}")
print(f"clean physics loss = {clean_phys_loss:.3f}")
print()
print(f"gsae OWT loss = {gsae_owt_loss:.3f}")
print(f"gsae physics loss = {gsae_phys_loss:.3f}")
print()
print(f"gsae OWT L0 = {gsae_owt_l0:.1f}")
print(f"gsae physics L0 = {gsae_phys_l0:.1f}")
print()
print(f"gsae OWT MSE = {gsae_owt_mse:.1f}")
print(f"gsae physics MSE = {gsae_phys_mse:.1f}")
print()
print(f"sparsest ssae OWT L0 = {ssae_owt_l0:.1f}")
print(f"sparsest ssae physics L0 = {ssae_phys_l0:.1f}")


clean OWT loss = 4.078
clean physics loss = 4.260

gsae OWT loss = 4.143
gsae physics loss = 4.740

gsae OWT L0 = 21.0
gsae physics L0 = 22.1

gsae OWT MSE = 144.1
gsae physics MSE = 168.5

sparsest ssae OWT L0 = 1.8
sparsest ssae physics L0 = 4.7


In [27]:
def get_l0_and_scores(sae_list_list, 
                      tokens, 
                      ceiling_loss, 
                      clean_loss, 
                      ceiling_mse,
                      num_tokens = 10_000):
    l0_list = []
    score_list = []
    varexp_list = []
    
    for sae_list in tqdm(sae_list_list):
        l0, loss, mse = get_l0_loss_mse(sae_list, tokens, num_tokens=num_tokens)
        l0_list.append(l0)

        # compute loss recovered score
        score = (ceiling_loss - loss) / (ceiling_loss - clean_loss)
        score_list.append(score)

        # compute variance explained
        varexp = 1 - mse / ceiling_mse
        varexp_list.append(varexp)
        
    return l0_list, score_list, varexp_list

In [28]:
num_tokens = 10_000
# get pareto data for SSAEs
print("0")
ssae_list_list = [[ssae, gsae] for ssae in ssae_list]
ssae_owt_l0, ssae_owt_scores, ssae_owt_varexp = get_l0_and_scores(ssae_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss, gsae_owt_mse, num_tokens=num_tokens)
ssae_phys_l0, ssae_phys_scores, ssae_phys_varexp = get_l0_and_scores(ssae_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss, gsae_phys_mse, num_tokens=num_tokens)
print(ssae_owt_scores, ssae_owt_l0)
# get pareto data for finetuned GSAEs
print("1")
gsae_fast_list_list = [[gsae] for gsae in gsae_0_list]
gsae_fast_owt_l0, gsae_fast_owt_scores, gsae_fast_owt_varexp = get_l0_and_scores(gsae_fast_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss, gsae_owt_mse, num_tokens=num_tokens)
gsae_fast_phys_l0, gsae_fast_phys_scores, gsae_fast_phys_varexp = get_l0_and_scores(gsae_fast_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss, gsae_phys_mse, num_tokens=num_tokens)

print("2")
gsae_medium_list_list = [[gsae] for gsae in gsae_0p1_list]
gsae_medium_owt_l0, gsae_medium_owt_scores, gsae_medium_owt_varexp = get_l0_and_scores(gsae_medium_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss, gsae_owt_mse, num_tokens=num_tokens)
gsae_medium_phys_l0, gsae_medium_phys_scores, gsae_medium_phys_varexp = get_l0_and_scores(gsae_medium_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss, gsae_phys_mse, num_tokens=num_tokens)

print("3")
gsae_slow_list_list = [[gsae] for gsae in gsae_0p5_list]
gsae_slow_owt_l0, gsae_slow_owt_scores, gsae_slow_owt_varexp = get_l0_and_scores(gsae_slow_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss, gsae_owt_mse, num_tokens=num_tokens)
gsae_slow_phys_l0, gsae_slow_phys_scores, gsae_slow_phys_varexp = get_l0_and_scores(gsae_slow_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss, gsae_phys_mse, num_tokens=num_tokens)

0


100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


[-1.0, -0.5, 0.0] [43.5, 33.1875, 23.0703125]
1


100%|██████████| 3/3 [00:02<00:00,  1.25it/s]


2


100%|██████████| 3/3 [00:02<00:00,  1.24it/s]


3


100%|██████████| 3/3 [00:02<00:00,  1.25it/s]


In [33]:
import pandas as pd
import plotly.express as px

# Create a DataFrame for each set of data
df_ssae = pd.DataFrame({'l0': ssae_owt_l0, 'Scores': ssae_owt_varexp, 'Type': 'SSAE'})
df_gsae_fast = pd.DataFrame({'l0': gsae_fast_owt_l0, 'Scores': gsae_fast_owt_varexp, 'Type': 'GSAE Fast'})
df_gsae_slow = pd.DataFrame({'l0': gsae_slow_owt_l0, 'Scores': gsae_slow_owt_varexp, 'Type': 'GSAE Slow'})
df_gsae_medium = pd.DataFrame({'l0': gsae_medium_owt_l0, 'Scores': gsae_medium_owt_varexp, 'Type': 'GSAE Medium'})

# Combine the DataFrames
df = pd.concat([df_ssae, df_gsae_fast, df_gsae_slow, df_gsae_medium])

# Create a line plot
fig = px.line(df, x='l0', y='Scores', color='Type', labels={'l0': 'l0', 'Scores': 'Scores'}, title='OWT Pareto Frontiers')

# Show the plot
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [37]:
import pandas as pd
import plotly.express as px

# Create a DataFrame for each set of data
df_ssae = pd.DataFrame({'l0': ssae_phys_l0, 'Scores': ssae_phys_varexp, 'Type': 'SSAE'})
df_gsae_fast = pd.DataFrame({'l0': gsae_fast_phys_l0, 'Scores': gsae_fast_phys_varexp, 'Type': 'GSAE Fast'})
df_gsae_slow = pd.DataFrame({'l0': gsae_slow_phys_l0, 'Scores': gsae_slow_phys_varexp, 'Type': 'GSAE Slow'})
df_gsae_medium = pd.DataFrame({'l0': gsae_medium_phys_l0, 'Scores': gsae_medium_phys_varexp, 'Type': 'GSAE Medium'})

# Combine the DataFrames
df = pd.concat([df_ssae, df_gsae_fast, df_gsae_slow, df_gsae_medium])

# Create a line plot
fig = px.line(df, x='l0', y='Scores', color='Type', labels={'l0': 'l0', 'Scores': 'Scores'}, title='Phys Pareto Frontiers')

# Show the plot
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

